In [1]:
import numpy as np
import pandas as pd
from pandasql import sqldf

In [2]:
df_orders = pd.read_csv('instacart-market-basket-analysis/order_products__prior.csv')

In [3]:
df_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [4]:
df_orders.shape

(32434489, 4)

In [92]:
len(df_orders.order_id.unique())

3214874

In [5]:
df_product = pd.read_csv("instacart-market-basket-analysis/products.csv")

In [6]:
df_product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [7]:
df_product.shape

(49688, 4)

In [8]:
df_dept = pd.read_csv("instacart-market-basket-analysis/departments.csv")

In [9]:
df_dept.shape

(21, 2)

In [10]:
df_dept.head(25)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [11]:
query = """
            select
                  a.order_id,
                  a.product_id,
                  b.department_id
            from
                df_orders a
            join
                df_product b
            on 
                a.product_id = b.product_id
                
        """

In [12]:
df_orders_detailed = sqldf(query, globals())

In [13]:
df_orders_detailed.head()

,order_id,product_id,department_id
0,2,33120,16
1,2,28985,4
2,2,9327,13
3,2,45918,13
4,2,30035,13


In [14]:
df_orders_detailed.shape

(32434489, 3)

In [15]:
df_items = df_orders_detailed[df_orders_detailed['department_id'].isin([1,3,4,7,13,14,16,19,20])]

In [16]:
df_items.shape

(27520600, 3)

In [18]:
query = """
            select
                  product_id1,
                  department_id1,
                  product_id2,
                  department_id2,
                  count(*) as cnt
            from
            (
                select a.product_id as product_id1,
                       a.department_id as department_id1,
                       b.product_id as product_id2,
                       b.department_id as department_id2
                from 
                       df_items a
                join
                       df_items b
                on 
                       a.order_id = b.order_id
            ) a
            where
                product_id1 < product_id2
            and 
                department_id1 == department_id2
            group by
                product_id1,
                department_id1,
                product_id2,
                department_id2
            order by cnt desc
                
        """

In [19]:
df_grouped = sqldf(query, globals())

In [24]:
df_grouped.head()

,product_id1,department_id1,product_id2,department_id2,cnt
0,13176,4,47209,4,62341
1,13176,4,21137,4,61628
2,21137,4,24852,4,56156
3,24852,4,47766,4,53395
4,21903,4,24852,4,51395


In [25]:
df_grouped.shape

(3483348, 5)

In [26]:
query = """
            select
                  a.product_id1,
                  a.product_id2,
                  a.product_name1,
                  b.product_name as product_name2,
                  a.department_id1,
                  a.department_id2,
                  a.cnt
            from
            (
                select
                    a.product_id1,
                    a.product_id2,
                    b.product_name as product_name1,
                    a.department_id1,
                    a.department_id2,
                    a.cnt
                from
                    (select * from df_grouped) a
                join
                    (select * from df_product) b
                on 
                    a.product_id1 = b.product_id
                and
                    a.department_id1 = b.department_id
            ) a
            join
                (select * from df_product) b
            on
                a.product_id2 = b.product_id
            and
                a.department_id2 = b.department_id
        """

In [27]:
df_grouped_detailed = sqldf(query,globals())

In [28]:
df_grouped_detailed.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,cnt
0,13176,47209,Bag of Organic Bananas,Organic Hass Avocado,4,4,62341
1,13176,21137,Bag of Organic Bananas,Organic Strawberries,4,4,61628
2,21137,24852,Organic Strawberries,Banana,4,4,56156
3,24852,47766,Banana,Organic Avocado,4,4,53395
4,21903,24852,Organic Baby Spinach,Banana,4,4,51395


In [29]:
df_grouped_detailed.tail()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,cnt
3483343,49621,49686,Challah Bread,Artisan Baguette,3,3,1
3483344,49630,49678,Cut Leaf Spinach,Large Chicken & Cheese Taquitos,1,1,1
3483345,49635,49683,"Baby Food Blueberry, Parsnip & Buckwheat Stage 2",Cucumber Kirby,4,4,1
3483346,49667,49685,Enchilada with Spanish Rice & Beans Meal,En Croute Roast Hazelnut Cranberry,1,1,1
3483347,49673,49678,Stone Baked Multi Grain Artisan Rolls,Large Chicken & Cheese Taquitos,1,1,1


In [30]:
df_grouped_detailed.shape

(3483348, 7)

In [31]:
df_grouped_detailed[(df_grouped_detailed['department_id1']!=4) & (df_grouped_detailed['department_id2']!=4)].shape

(3160834, 7)

In [32]:
df_grouped_detailed[(df_grouped_detailed['department_id1']!=4) & (df_grouped_detailed['department_id2']!=4)].head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,cnt
116,35221,44632,Lime Sparkling Water,Sparkling Water Grapefruit,7,7,13284
251,4957,33754,Total 2% Lowfat Greek Strained Yogurt With Blu...,Total 2% with Strawberry Lowfat Greek Strained...,16,16,9331
258,21709,44632,Sparkling Lemon Water,Sparkling Water Grapefruit,7,7,9224
275,22035,27845,Organic Whole String Cheese,Organic Whole Milk,16,16,8769
286,21709,35221,Sparkling Lemon Water,Lime Sparkling Water,7,7,8546


In [33]:
query = """
            select 
                product_id,
                count(*) as cnt
            from
                df_orders
            group by
                product_id
        """

In [34]:
df_product_bought_count = sqldf(query,globals())

In [35]:
df_product_bought_count.head()

,product_id,cnt
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [36]:
df_grouped_detailed_sample = df_grouped_detailed.head(50)

In [38]:
df_grouped_detailed_sample.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,cnt
0,13176,47209,Bag of Organic Bananas,Organic Hass Avocado,4,4,62341
1,13176,21137,Bag of Organic Bananas,Organic Strawberries,4,4,61628
2,21137,24852,Organic Strawberries,Banana,4,4,56156
3,24852,47766,Banana,Organic Avocado,4,4,53395
4,21903,24852,Organic Baby Spinach,Banana,4,4,51395


In [39]:
df_product_bought_count_sample = df_product_bought_count.head(10)

In [40]:
df_product_bought_count_sample.head()

,product_id,cnt
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [41]:
query = """
            select 
                product_id1,
                product_id2,
                product_name1,
                product_name2,
                department_id1,
                department_id2,
                cnt as item_intersection,
                cnt1 as item1_count,
                cnt2 as item2_count,
                cnt1+cnt2-cnt as item_union,
                (cnt*1.0)/(cnt1+cnt2-cnt) as ratio
                from
               (select 
                    a.*,
                    b.cnt as cnt2
                from
                    (select 
                        a.*,
                        b.cnt as cnt1
                    from
                        (select * from df_grouped_detailed_sample) a
                    join
                        (select * from df_product_bought_count_sample) b
                    on
                        a.product_id1 = b.product_id) a
                join
                    (select * from df_product_bought_count_sample) b
                on
                    a.product_id2 = b.product_id) a
        """
df_tmp = sqldf(query,globals())

In [42]:
df_tmp.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,item_intersection,item1_count,item2_count,item_union,ratio


In [73]:
query = """
            select 
                product_id1,
                product_id2,
                product_name1,
                product_name2,
                department_id1,
                department_id2,
                cnt as item_intersection,
                cnt1 as item1_count,
                cnt2 as item2_count,
                cnt1+cnt2-cnt as item_union,
                (cnt*1.0)/min(cnt1,cnt2) as ratio
                from
                    (select 
                        a.*,
                        b.cnt as cnt2
                    from
                        (select 
                            a.*,
                            b.cnt as cnt1
                        from
                            (select * from df_grouped_detailed) a
                        join
                            (select * from df_product_bought_count) b
                        on
                            a.product_id1 = b.product_id) a
                    join
                        (select * from df_product_bought_count) b
                    on
                        a.product_id2 = b.product_id) a

        """

In [74]:
df_ratio = sqldf(query,globals())

In [75]:
df_ratio.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,item_intersection,item1_count,item2_count,item_union,ratio
0,13176,47209,Bag of Organic Bananas,Organic Hass Avocado,4,4,62341,379450,213584,530693,0.291880
1,13176,21137,Bag of Organic Bananas,Organic Strawberries,4,4,61628,379450,264683,582505,0.232837
2,21137,24852,Organic Strawberries,Banana,4,4,56156,264683,472565,681092,0.212163
3,24852,47766,Banana,Organic Avocado,4,4,53395,472565,176815,595985,0.301982
4,21903,24852,Organic Baby Spinach,Banana,4,4,51395,241921,472565,663091,0.212445


In [79]:
df_ratio_sorted = df_ratio.sort_values('ratio',ascending=True)

In [80]:
df_ratio_sorted.head()

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,item_intersection,item1_count,item2_count,item_union,ratio
2188775,6184,48679,Clementines,Organic Garnet Sweet Potato (Yam),4,4,1,30843,51939,82781,0.000032
2188766,6184,44910,Clementines,Organic Spring Mix Salad,4,4,1,30843,21416,52258,0.000047
2188740,6184,38159,Clementines,Organic Yellow Peaches,4,4,1,30843,21301,52143,0.000047
3090221,25931,38383,Organic Baby Spinach Salad,Organic White Onions,4,4,1,17672,23641,41312,0.000057
1898390,1463,40174,Organic Milk,2% Reduced Fat Organic Milk,16,16,1,27992,16356,44347,0.000061


In [84]:
df_ratio_sorted[(df_ratio_sorted.item1_count > 200) & (df_ratio_sorted.item2_count > 200) 
               & (df_ratio_sorted.item_intersection > 5) & (df_ratio_sorted.product_id1!=6184)].head(50)

,product_id1,product_id2,product_name1,product_name2,department_id1,department_id2,item_intersection,item1_count,item2_count,item_union,ratio
670569,32478,41665,Reduced Fat 2% Milk,Organic Mexican Blend Finely Shredded Cheese,16,16,6,17320,18367,35681,0.000346
594316,196,45603,Soda,Trilogy Kombucha Drink,7,7,6,35791,16791,52576,0.000357
349237,11520,30233,Large Alfresco Eggs,Organic Large Brown Grade AA Cage Free Eggs,16,16,12,40376,32105,72469,0.000374
512146,23909,29447,2% Reduced Fat Milk,"Milk, Organic, Vitamin D",16,16,8,37091,20198,57281,0.000396
531533,2078,32478,Unsalted Pure Irish Butter,Reduced Fat 2% Milk,16,16,7,19846,17320,37159,0.000404
635584,14084,19677,Organic Unsweetened Vanilla Almond Milk,Original Pure Creamy Almond Milk,16,16,6,15935,14712,30641,0.000408
633303,13249,32478,Skim Milk,Reduced Fat 2% Milk,16,16,6,14289,17320,31603,0.000420
640525,16083,18465,Organic Large Brown Eggs,Organic Grade A Free Range Large Brown Eggs,16,16,6,14255,40045,54294,0.000421
430860,196,30776,Soda,Organic Raw Kombucha Gingerade,7,7,9,35791,20707,56489,0.000435
644443,17795,30827,Organic Green Leaf Lettuce,Seedless Cucumbers,4,4,6,14016,13571,27581,0.000442


In [78]:
# !jupyter nbextension enable --py --sys-prefix qgrid
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [57]:
# import qgrid

In [58]:
# qgrid.show_grid(df_ratio_sorted.head(50),
#                 grid_options={'fullWidthRows': True,
#                               'syncColumnCellResize': True,
#                               'forceFitColumns': True,
#                               'rowHeight': 40,
#                               'enableColumnReorder': True,
#                               'enableTextSelectionOnCells': True,
#                               'editable': True})

In [81]:
# df_ratio_sorted[(df_ratio_sorted.item1_count > 200) & (df_ratio_sorted.item2_count > 200) 
#                & ((df_ratio_sorted.item1_count*1.0)/(df_ratio_sorted.item2_count*1.0) > 0.40) &
#                ((df_ratio_sorted.item1_count*1.0)/(df_ratio_sorted.item2_count*1.0) < 0.60)
#                & (df_ratio_sorted.item_intersection > 20)].head(50)

In [177]:
# df_ratio_sorted[df_ratio_sorted.item_union > 500].head(50).to_csv("r.csv",header=True,index=False)

In [ ]:
# df_ratio_sorted

In [186]:
# df_product[df_product['product_id']==13269]

In [187]:
# df_product[df_product['product_id']==44786]

In [95]:
df_ratio_sorted[(df_ratio_sorted.item1_count > 200) & (df_ratio_sorted.item2_count > 200) 
               & (df_ratio_sorted.item_intersection > 5) & (df_ratio_sorted.product_id1!=6184)].head(50).to_csv('s.csv',header=True,index=False)